# Splitting sequences into ORF and barcode parts

After trimming the backbone sequences and reverse complementing the reverse complement sequences, all sequences have this structure:

**5'-[ORF]nnn[Barcode]-3'**

Where **nnn** represents the internal linker:

`AACCCAGCTTTCTTGTACAAAGTGGTTTAATGAGTTTAAACCTCGAGCGTAACTATAACGGTCCTAAGGTAGCGAA`

Goal: split sequences into ORF and barcode parts, and add the barcode to the ORF name in the FASTQ header.

Approach: use `porechop_abi` to split the sequences based on the internal linker sequence, and a custom Python script to move the barcodes to the read name in the FASTQ header of the ORF sequences.

## Testing the `porechop_abi` approach

First test the `porechop_abi` approach on a small subset of the data to make sure it works as expected. Use the `cutadapt_test_04.fastq` file from the previous step, which has been optimised for trimming of backbone sequences, and contains 56.2% of reads with adapters.

internal_sequence.txt:
```
>Internal_linker_Middle
AACCCAGCTTTCTTGTACAAAGTGGTTTAATGAGTTTAAACCTCGAGCGTAACTATAACGGTCCTAAGGTAGCGAA

```

Explanation of the `porechop_abi` arguments:
- `--custom_adapters`: specifies the file containing custom adapter sequences to be used for trimming.
- `--discard_database`: discards the internal database of adapter sequences, so only custom adapters are used.
- `--threads 4`: uses 4 threads for parallel processing.
- `--middle_threshold 75`: sets the minimum similarity threshold (in percentage) for detecting middle adapters.
- `--end_threshold 95`: sets the minimum similarity threshold (in percentage) for detecting end adapters.
- `--extra_middle_trim_good_side 0`: specifies how many base pairs to trim from the good side of middle adapters (0 means no trimming).
- `--extra_middle_trim_bad_side 0`: specifies how many base pairs to trim from the bad side of middle adapters (0 means no trimming).
- `--min_split_read_size 24`: sets the minimum size of a read that is considered for splitting (24 is the barcode length).
- `--end_threshold 101`: sets the maximum allowed end threshold value (101% in this case, i.e. disable end trimming).

In [6]:
%%bash

DATA_DIR="/media/niek/4TB_SSD2/analyses/GPS_ONT"
porechop_abi -i $DATA_DIR/cutadapt_test_04.fastq \
    -o $DATA_DIR/split_trimmed_test_01.fastq \
    --custom_adapters internal_sequence.txt \
    --discard_database \
    --threads 4 \
    --middle_threshold 95 \
    --end_threshold 95 \
	--extra_middle_trim_good_side 0 \
    --extra_middle_trim_bad_side 0 \
	--min_split_read_size 24 \
	--end_threshold 101 \
    > porechop_abi_test_01.log


Important info from log:

```shell
    0 / 5,621 reads had adapters trimmed from their start (0 bp removed)
    0 / 5,621 reads had adapters trimmed from their end (0 bp removed)
    ...
    5,614 / 5,621 reads were split based on middle adapters
```

5614 out of 5621 (99.89%) reads were split based on the middle adapter, which is the internal linker sequence. 

This is a good result, as we expect most reads to contain the internal linker sequence, and therefore be split into ORF and barcode parts.

------

Now run the `porechop_abi` command on the full dataset to split all sequences into ORF and barcode parts, and add the barcode to the ORF name in the FASTQ header:

```shell
#!/bin/bash

#SBATCH -A JNATHAN-SL2-CPU
#SBATCH --mail-type=BEGIN,END,FAIL
#SBATCH -p cclake
#SBATCH -D /home/nw416/rds/rds-jan_1-tpuFdqHBAEk/gps_ont
#SBATCH -o ont_split.log
#SBATCH -c 8
#SBATCH -t 06:00:00
#SBATCH -J ont_split
#SBATCH --mem=60G

# Initialize Conda for script usage
source "/home/nw416/miniforge3/etc/profile.d/conda.sh"
conda activate ont

DATA_DIR=/home/nw416/rds/rds-jan_1-tpuFdqHBAEk/gps_ont
TRIMMED_FASTQ=$DATA_DIR/trimmed.fastq
SPLIT_FASTQ=$DATA_DIR/split_trimmed.fastq

# Split sequences into ORF and barcode sequences 
# ---------------------------------------
porechop_abi -i $TRIMMED_FASTQ \
    -o $SPLIT_FASTQ \
    --custom_adapters internal_sequence.txt \
    --discard_database \
    --threads 8 \
    --middle_threshold 95 \
    --end_threshold 95 \
	--extra_middle_trim_good_side 0 \
    --extra_middle_trim_bad_side 0 \
	--min_split_read_size 24 \
	--end_threshold 101 \
    > split.log
```
File name: 03_split.sh

------

While this approach works well on a small file, it is not suitable for the full dataset, as it is extremely slow, due to the adapter trimming step that still takes place. It also takes a hugh amount of memory. Even with 60G allocated it runs out of memory and crashes after processing less than 1% of the data. This also took 30 min so we will never be able to run this on the HPC.

## Using `cutadapt` to split sequences into ORF and barcode parts

Since the `porechop_abi` approach is not suitable for the full dataset, we will use `cutadapt` to split the sequences into ORF and barcode parts based on the internal linker sequence. 

This will be more efficient and faster than the `porechop_abi`:

In [ ]:
%%bash
DATA_DIR="/media/niek/4TB_SSD2/analyses/GPS_ONT"
TRIMMED_FASTQ="$DATA_DIR/cutadapt_test_06.fastq"

cutadapt \
	-a AACCCAGCTTTCTTGTACAAAGTGGTTTAATGAGTTTAAACCTCGAGCGTAACTATAACGGTCCTAAGGTAGCGAACCAGTAGGTCCACTATGAGT \
	-e 0.15 \
	-O 40 \
	--untrimmed-output $DATA_DIR/trimmed_unsplit_01.fastq \
	--rest-file $DATA_DIR/barcode_01.txt \
	-o $DATA_DIR/orf_01.fastq \
	$TRIMMED_FASTQ > cutadapt_split_test_01.log


Explanation of the `cutadapt` arguments and output files:
- `-a`: specifies the 3' adapter sequence to be trimmed.
- `-e`: sets the maximum error rate (0.15 = 15%).
- `-O`: sets the minimum overlap length (40 nucleotides).
- `--rest-file`: outputs sequences that came after the adapter sequence (i.e. the barcode).
- `--untrimmed-output`: outputs sequences that do not match any adapter to FASTQ.
- `-o`: trimmed sequences (i.e. clean ORF sequences) in FASTQ format.

The format of the rest file is:

```text
TAATAGGCTGGGAGTGGTCAGTCC 00a2870a-79fb-4e1e-a961-34da5bd0a6fd runid=397513e0-9874-4c38-b5d7-80582efe53ad ch=2882 start_time=2026-02-14T16:44:08.514475-08:00 flow_cell_id=PBK43160 basecall_gpu=NVIDIA_A100_80GB_PCIe protocol_group_id=260213EUG_CUSTOM sample_id=TIMMS parent_read_id=00a2870a-79fb-4e1e-a961-34da5bd0a6fd basecall_model_version_id=dna_r10.4.1_e8.2_400bps_sup@v4.3.0 rc
```

Each line has the rest sequence (i.e. barcode) and read name separated by a space.

Cut adapt log (relevant parts):

```shell
This is cutadapt 5.2 with Python 3.12.12
Command line parameters: -a AACCCAGCTTTCTTGTACAAAGTGGTTTAATGAGTTTAAACCTCGAGCGTAACTATAACGGTCCTAAGGTAGCGAACCAGTAGGTCCACTATGAGT -e 0.15 -O 40 --untrimmed-output /media/niek/4TB_SSD2/analyses/GPS_ONT/trimmed_unsplit_01.fastq --rest-file /media/niek/4TB_SSD2/analyses/GPS_ONT/barcode_01.fastq -o /media/niek/4TB_SSD2/analyses/GPS_ONT/orf_01.fastq /media/niek/4TB_SSD2/analyses/GPS_ONT/cutadapt_test_06.fastq
Processing single-end reads on 1 core ...

=== Summary ===

Total reads processed:                   5,487
Reads with adapters:                     5,441 (99.2%)

== Read fate breakdown ==
Reads discarded as untrimmed:               46 (0.8%)
Reads written (passing filters):         5,441 (99.2%)

Total basepairs processed:     5,621,338 bp
Total written (filtered):      4,909,875 bp (87.3%)

=== Adapter 1 ===

Sequence: AACCCAGCTTTCTTGTACAAAGTGGTTTAATGAGTTTAAACCTCGAGCGTAACTATAACGGTCCTAAGGTAGCGAACCAGTAGGTCCACTATGAGT; Type: regular 3'; Length: 96; Trimmed: 5441 times

Minimum overlap: 40
No. of allowed errors:
1-5 bp: 0; 6-12 bp: 1; 13-19 bp: 2; 20-25 bp: 3; 26-32 bp: 4; 33-39 bp: 5; 40-45 bp: 6; 46-52 bp: 7; 53-59 bp: 8; 60-65 bp: 9; 66-72 bp: 10; 73-79 bp: 11; 80-85 bp: 12; 86-92 bp: 13; 93-96 bp: 14

Bases preceding removed adapters:
  A: 7.2%
  C: 16.9%
  G: 67.1%
  T: 8.9%
  none/other: 0.0%
```

Seems to have worked well, as nearly all reads (99.2%) had the adapter sequence and were split.

One possibility is that the options were too promiscuous when matching the internal linker.

Also try a more stringent approach:

In [9]:
%%bash
DATA_DIR="/media/niek/4TB_SSD2/analyses/GPS_ONT"
TRIMMED_FASTQ="$DATA_DIR/cutadapt_test_06.fastq"

cutadapt \
	-a AACCCAGCTTTCTTGTACAAAGTGGTTTAATGAGTTTAAACCTCGAGCGTAACTATAACGGTCCTAAGGTAGCGAACCAGTAGGTCCACTATGAGT \
	-e 0.15 \
	-O 96 \
	--untrimmed-output $DATA_DIR/trimmed_unsplit_02.fastq \
	--rest-file $DATA_DIR/barcode_02.txt \
	-o $DATA_DIR/orf_02.fastq \
	$TRIMMED_FASTQ > cutadapt_split_test_02.log

```shell
This is cutadapt 5.2 with Python 3.12.12
Command line parameters: -a AACCCAGCTTTCTTGTACAAAGTGGTTTAATGAGTTTAAACCTCGAGCGTAACTATAACGGTCCTAAGGTAGCGAACCAGTAGGTCCACTATGAGT -e 0.15 -O 96 --untrimmed-output /media/niek/4TB_SSD2/analyses/GPS_ONT/trimmed_unsplit_02.fastq --rest-file /media/niek/4TB_SSD2/analyses/GPS_ONT/barcode_02.txt -o /media/niek/4TB_SSD2/analyses/GPS_ONT/orf_02.fastq /media/niek/4TB_SSD2/analyses/GPS_ONT/cutadapt_test_06.fastq
Processing single-end reads on 1 core ...

=== Summary ===

Total reads processed:                   5,487
Reads with adapters:                     5,441 (99.2%)

== Read fate breakdown ==
Reads discarded as untrimmed:               46 (0.8%)
Reads written (passing filters):         5,441 (99.2%)

Total basepairs processed:     5,621,338 bp
Total written (filtered):      4,909,875 bp (87.3%)

=== Adapter 1 ===

Sequence: AACCCAGCTTTCTTGTACAAAGTGGTTTAATGAGTTTAAACCTCGAGCGTAACTATAACGGTCCTAAGGTAGCGAACCAGTAGGTCCACTATGAGT; Type: regular 3'; Length: 96; Trimmed: 5441 times

Minimum overlap: 96
No. of allowed errors:
1-5 bp: 0; 6-12 bp: 1; 13-19 bp: 2; 20-25 bp: 3; 26-32 bp: 4; 33-39 bp: 5; 40-45 bp: 6; 46-52 bp: 7; 53-59 bp: 8; 60-65 bp: 9; 66-72 bp: 10; 73-79 bp: 11; 80-85 bp: 12; 86-92 bp: 13; 93-96 bp: 14

Bases preceding removed adapters:
  A: 7.2%
  C: 16.9%
  G: 67.1%
  T: 8.9%
  none/other: 0.0%
```

Increasing `-O` to 96 did not change the results. This is a good sign. Try to decrease the error rate to 0.1 to see if this changes the results:

In [10]:
%%bash
DATA_DIR="/media/niek/4TB_SSD2/analyses/GPS_ONT"
TRIMMED_FASTQ="$DATA_DIR/cutadapt_test_06.fastq"

cutadapt \
	-a AACCCAGCTTTCTTGTACAAAGTGGTTTAATGAGTTTAAACCTCGAGCGTAACTATAACGGTCCTAAGGTAGCGAACCAGTAGGTCCACTATGAGT \
	-e 0.1 \
	-O 96 \
	--untrimmed-output $DATA_DIR/trimmed_unsplit_03.fastq \
	--rest-file $DATA_DIR/barcode_03.txt \
	-o $DATA_DIR/orf_03.fastq \
	$TRIMMED_FASTQ > cutadapt_split_test_03.log

```shell
This is cutadapt 5.2 with Python 3.12.12
Command line parameters: -a AACCCAGCTTTCTTGTACAAAGTGGTTTAATGAGTTTAAACCTCGAGCGTAACTATAACGGTCCTAAGGTAGCGAACCAGTAGGTCCACTATGAGT -e 0.1 -O 96 --untrimmed-output /media/niek/4TB_SSD2/analyses/GPS_ONT/trimmed_unsplit_03.fastq --rest-file /media/niek/4TB_SSD2/analyses/GPS_ONT/barcode_03.txt -o /media/niek/4TB_SSD2/analyses/GPS_ONT/orf_03.fastq /media/niek/4TB_SSD2/analyses/GPS_ONT/cutadapt_test_06.fastq
Processing single-end reads on 1 core ...

=== Summary ===

Total reads processed:                   5,487
Reads with adapters:                     5,407 (98.5%)

== Read fate breakdown ==
Reads discarded as untrimmed:               80 (1.5%)
Reads written (passing filters):         5,407 (98.5%)

Total basepairs processed:     5,621,338 bp
Total written (filtered):      4,882,040 bp (86.8%)

=== Adapter 1 ===

Sequence: AACCCAGCTTTCTTGTACAAAGTGGTTTAATGAGTTTAAACCTCGAGCGTAACTATAACGGTCCTAAGGTAGCGAACCAGTAGGTCCACTATGAGT; Type: regular 3'; Length: 96; Trimmed: 5407 times

Minimum overlap: 96
No. of allowed errors:
1-9 bp: 0; 10-19 bp: 1; 20-29 bp: 2; 30-39 bp: 3; 40-49 bp: 4; 50-59 bp: 5; 60-69 bp: 6; 70-79 bp: 7; 80-89 bp: 8; 90-96 bp: 9

Bases preceding removed adapters:
  A: 7.1%
  C: 16.8%
  G: 67.2%
  T: 8.9%
  none/other: 0.0%
```
Slight reduction in the number of reads with adapters (98.5% vs 99.2%), but still a good result. 

One advantage of using a more stringent error rate is that it takes less time to run, as there are fewer reads that need to be processed.

Try one more time with an even more stringent error rate of 0.05:


In [11]:
%%bash
DATA_DIR="/media/niek/4TB_SSD2/analyses/GPS_ONT"
TRIMMED_FASTQ="$DATA_DIR/cutadapt_test_06.fastq"

cutadapt \
	-a AACCCAGCTTTCTTGTACAAAGTGGTTTAATGAGTTTAAACCTCGAGCGTAACTATAACGGTCCTAAGGTAGCGAACCAGTAGGTCCACTATGAGT \
	-e 0.05 \
	-O 96 \
	--untrimmed-output $DATA_DIR/trimmed_unsplit_04.fastq \
	--rest-file $DATA_DIR/barcode_04.txt \
	-o $DATA_DIR/orf_04.fastq \
	$TRIMMED_FASTQ > cutadapt_split_test_04.log

```shell
This is cutadapt 5.2 with Python 3.12.12
Command line parameters: -a AACCCAGCTTTCTTGTACAAAGTGGTTTAATGAGTTTAAACCTCGAGCGTAACTATAACGGTCCTAAGGTAGCGAACCAGTAGGTCCACTATGAGT -e 0.05 -O 96 --untrimmed-output /media/niek/4TB_SSD2/analyses/GPS_ONT/trimmed_unsplit_04.fastq --rest-file /media/niek/4TB_SSD2/analyses/GPS_ONT/barcode_04.txt -o /media/niek/4TB_SSD2/analyses/GPS_ONT/orf_04.fastq /media/niek/4TB_SSD2/analyses/GPS_ONT/cutadapt_test_06.fastq
Processing single-end reads on 1 core ...

=== Summary ===

Total reads processed:                   5,487
Reads with adapters:                     5,007 (91.3%)

== Read fate breakdown ==
Reads discarded as untrimmed:              480 (8.7%)
Reads written (passing filters):         5,007 (91.3%)

Total basepairs processed:     5,621,338 bp
Total written (filtered):      4,546,224 bp (80.9%)

=== Adapter 1 ===

Sequence: AACCCAGCTTTCTTGTACAAAGTGGTTTAATGAGTTTAAACCTCGAGCGTAACTATAACGGTCCTAAGGTAGCGAACCAGTAGGTCCACTATGAGT; Type: regular 3'; Length: 96; Trimmed: 5007 times

Minimum overlap: 96
No. of allowed errors:
1-19 bp: 0; 20-39 bp: 1; 40-59 bp: 2; 60-79 bp: 3; 80-96 bp: 4

Bases preceding removed adapters:
  A: 6.6%
  C: 16.3%
  G: 69.1%
  T: 8.0%
  none/other: 0.0%
```

This is a significant reduction in the number of reads with adapters (91.3% vs 98.5%).

The error rate of 0.05 translates to 4 errors allowed for a 96 bp overlap, which is still higher than the sequencing error rate (3% for Q15 and 1% for Q20). 

Therefore, this is a good balance between stringency and sensitivity, and we will use these parameters for the full dataset.

--------

SLURM script to run `cutadapt` on the full dataset:

```shell
#!/bin/bash

#SBATCH -A JNATHAN-SL2-CPU
#SBATCH --mail-type=BEGIN,END,FAIL
#SBATCH -p cclake
#SBATCH -D /home/nw416/rds/rds-jan_1-tpuFdqHBAEk/gps_ont
#SBATCH -o split_slurm.log
#SBATCH -c 8
#SBATCH -t 01:00:00
#SBATCH -J split
#SBATCH --mem=16G

# Initialize Conda for script usage
source "/home/nw416/miniforge3/etc/profile.d/conda.sh"
conda activate ont

DATA_DIR=/home/nw416/rds/rds-jan_1-tpuFdqHBAEk/gps_ont
TRIMMED_FASTQ=$DATA_DIR/trimmed.fastq.gz
UNSPLIT_FASTQ=$DATA_DIR/trimmed_unsplit.fastq.gz
BARCODES=$DATA_DIR/barcodes.txt
ORFS=$DATA_DIR/orfs.fastq.gz

cutadapt \
	-a AACCCAGCTTTCTTGTACAAAGTGGTTTAATGAGTTTAAACCTCGAGCGTAACTATAACGGTCCTAAGGTAGCGAACCAGTAGGTCCACTATGAGT \
	-e 0.05 \
	-O 96 \
	-j 8 \
	--untrimmed-output $UNSPLIT_FASTQ \
	--rest-file $BARCODES \
	-o $ORFS \
	$TRIMMED_FASTQ > split.log
```

--------

File name: 03_split.sh

Relevant parts of the log:

```shell
This is cutadapt 5.2 with Python 3.12.12
Command line parameters: -a AACCCAGCTTTCTTGTACAAAGTGGTTTAATGAGTTTAAACCTCGAGCGTAACTATAACGGTCCTAAGGTAGCGAACCAGTAGGTCCACTATGAGT -e 0.05 -O 96 -j 8 --untrimmed-output /home/nw416/rds/rds-jan_1-tpuFdqHBAEk/gps_ont/trimmed_unsplit.fastq.gz --rest-file /home/nw416/rds/rds-jan_1-tpuFdqHBAEk/gps_ont/barcodes.txt -o /home/nw416/rds/rds-jan_1-tpuFdqHBAEk/gps_ont/orfs.fastq.gz /home/nw416/rds/rds-jan_1-tpuFdqHBAEk/gps_ont/trimmed.fastq.gz
Processing single-end reads on 8 cores ...

=== Summary ===

Total reads processed:              13,329,423
Reads with adapters:                12,262,199 (92.0%)

== Read fate breakdown ==
Reads discarded as untrimmed:        1,067,224 (8.0%)
Reads written (passing filters):    12,262,199 (92.0%)

Total basepairs processed: 13,622,666,905 bp
Total written (filtered):  11,026,528,420 bp (80.9%)

=== Adapter 1 ===

Sequence: AACCCAGCTTTCTTGTACAAAGTGGTTTAATGAGTTTAAACCTCGAGCGTAACTATAACGGTCCTAAGGTAGCGAACCAGTAGGTCCACTATGAGT; Type: regular 3'; Length: 96; Trimmed: 12262199 times

Minimum overlap: 96
No. of allowed errors:
1-19 bp: 0; 20-39 bp: 1; 40-59 bp: 2; 60-79 bp: 3; 80-96 bp: 4

Bases preceding removed adapters:
  A: 7.0%
  C: 15.2%
  G: 70.7%
  T: 7.1%
  none/other: 0.0%
```

The output of the rest file only contains the barcode sequence and read name. It will be moved to the FASTQ header of the ORF sequences in the next step. 

The drawback of the rest file is that it does not contain the quality scores of the barcode sequences, which could be useful later for potential troubleshooting.

Use the following Python script to convert the rest file into FASTQ format:

--------

```python
import sys
import gzip
from Bio import SeqIO

# Usage: python extract_barcodes.py barcodes.txt trimmed.fastq.gz output_barcodes.fastq.gz

rest_file = sys.argv[1]
original_fastq = sys.argv[2]
output_file = sys.argv[3]

# Map Read ID -> Barcode Sequence
barcode_map = {}
with open(rest_file, "r") as f:
    for line in f:
        parts = line.strip().split(maxsplit=1)
        if len(parts) == 2:
            seq, header = parts
            read_id = header.split()[0]  # Get the UUID
            barcode_map[read_id] = seq

# Extract FASTQ records
open_func = gzip.open if original_fastq.endswith(".gz") else open
out_func = gzip.open if output_file.endswith(".gz") else open

with open_func(original_fastq, "rt") as in_handle, out_func(
    output_file, "wt"
) as out_handle:
    for record in SeqIO.parse(in_handle, "fastq"):
        if record.id in barcode_map:
            target_seq = barcode_map[record.id]
            # Find where the barcode starts in the original read
            # We search from the end to be safe with ONT noise
            start_pos = record.seq.rfind(target_seq)

            if start_pos != -1:
                # Slice the record (this automatically slices quality scores too!)
                barcode_record = record[start_pos:]
                SeqIO.write(barcode_record, out_handle, "fastq")

print(f"Done! Processed {len(barcode_map)} barcodes.")
```

--------

File name: extract_barcodes.py

First test with the small subset of the data generated earlier to make sure it works as expected:

In [12]:
%%bash
DATA_DIR="/media/niek/4TB_SSD2/analyses/GPS_ONT"
REST_FILE=$DATA_DIR/barcode_04.txt
TRIMMED_FASTQ="$DATA_DIR/cutadapt_test_06.fastq"
OUTPUT_FILE="$DATA_DIR/barcode_04_extracted.fastq"

python extract_barcodes.py $REST_FILE $TRIMMED_FASTQ $OUTPUT_FILE

Done! Processed 5006 barcodes.


Check first line of the rest file:
```
TAATAGGCTGGGAGTGGTCAGTCC 00a2870a-79fb-4e1e-a961-34da5bd0a6fd runid=397513e0-9874-4c38-b5d7-80582efe53ad ch=2882 start_time=2026-02-14T16:44:08.514475-08:00 flow_cell_id=PBK43160 basecall_gpu=NVIDIA_A100_80GB_PCIe protocol_group_id=260213EUG_CUSTOM sample_id=TIMMS parent_read_id=00a2870a-79fb-4e1e-a961-34da5bd0a6fd basecall_model_version_id=dna_r10.4.1_e8.2_400bps_sup@v4.3.0 rc
```

Check first record in barcode FASTQ file:
```
@00a2870a-79fb-4e1e-a961-34da5bd0a6fd runid=397513e0-9874-4c38-b5d7-80582efe53ad ch=2882 start_time=2026-02-14T16:44:08.514475-08:00 flow_cell_id=PBK43160 basecall_gpu=NVIDIA_A100_80GB_PCIe protocol_group_id=260213EUG_CUSTOM sample_id=TIMMS parent_read_id=00a2870a-79fb-4e1e-a961-34da5bd0a6fd basecall_model_version_id=dna_r10.4.1_e8.2_400bps_sup@v4.3.0 rc
TAATAGGCTGGGAGTGGTCAGTCC
+
JGNILKSJ0DCEJGFKGJNJGHHI
```

Check first record in trimmed FASTQ file:
```
@00a2870a-79fb-4e1e-a961-34da5bd0a6fd runid=397513e0-9874-4c38-b5d7-80582efe53ad ch=2882 start_time=2026-02-14T16:44:08.514475-08:00 flow_cell_id=PBK43160 basecall_gpu=NVIDIA_A100_80GB_PCIe protocol_group_id=260213EUG_CUSTOM sample_id=TIMMS parent_read_id=00a2870a-79fb-4e1e-a961-34da5bd0a6fd basecall_model_version_id=dna_r10.4.1_e8.2_400bps_sup@v4.3.0 rc
GAATATCATCCTGATTTAGAAAATTTGGATGAAGATGGATATACTCAATTACACTTCGACTCTCAAAGCAATACCAGGATAGCTGTTGTTTCAGAGAAAGGATTGTGTGCTGCATCTCCTCCTTGGCGCCTCATTGCTGTAATTTTGGGAATCCTATGCTTGGTAATACTGGTGATAGCTGTGGTCCTGGGTACCATGGGGGTTCTTTCCAGCCCTTGTCCTCCTAATTGGATTATATATGAGAAGAGCTGTTATCTATTCAGCATGTCACTAAATTCCTGGGATGGAAGTAAAAGACAATGCTGGCAACTGGGCTCTAATCTCCTAAAGATAGACAGCTCAAATGAATTGGTAAGTGTAGACTTCTGTTATGATTATCTGTGGTGTGTATCCTAGAACCCAGCTTTCTTGTACAAAGTGGTTTAATGAGTTTAAACCTCGAGCGTAACTATAACGGTCCTAAGGTAGCGAACCAGTAGGTCCACTATGAGTTAATAGGCTGGGAGTGGTCAGTCC
+
KGJHHGFCDDDESHKIOSRPFFEEGFHGSISIKJPKNKFFIDEFFIJMHGJGHIJGHJIKJIMHKPGGGGGEEFFGFFEDFEGGGHLPRONIKSMGNHH(((((BFGCCEDSLHJHKILHHHGHGIIPSSPQNJJOISRKROLIFBJHKNMMMLLSNINKIJNMKQLHSJIKKNKJNJJNLMMKRPGFHICGNSLSMSNHJJSPQNOQIMMKOSMSPLJGKKKSGGFHHJGJGLHPILKQQNNSSJJQMKSLSOKJKIJGHMJKSLLNOLPPNLNKJEHDHIJJNNPSJSOSSNJNNKMFGF88876@DDCGGIHGGJGMSSJLLHSMSPOLOLPIHNOKKJJQNSMIKKJIQIOIILHIKOLJLISKKQHIIPNMOISOSQNLPSLSLIHIMOSNLSIHHMEACBAFFEKJLISNMKLLNRSKSSKSNOSSSROJIHH@@@@AMJKLPQSHHHFGG@@=>FOHHHGEFIKGHIKIJJJMPIGJJHSLIHJGJGNILKSJ0DCEJGFKGJNJGHHI
```

Sequence and scores match between barcode FASTQ and trimmed FASTQ files, which means the script works as expected.

Now run on HPC with full dataset:

--------

```shell
#!/bin/bash

#SBATCH -A JNATHAN-SL2-CPU
#SBATCH --mail-type=BEGIN,END,FAIL
#SBATCH -p cclake
#SBATCH -D /home/nw416/rds/rds-jan_1-tpuFdqHBAEk/gps_ont
#SBATCH -o barcode_fq.log
#SBATCH -c 4
#SBATCH -t 02:00:00
#SBATCH -J barcode_fq
#SBATCH --mem=8G

# Initialize Conda for script usage
source "/home/nw416/miniforge3/etc/profile.d/conda.sh"
conda activate ont

DATA_DIR=/home/nw416/rds/rds-jan_1-tpuFdqHBAEk/gps_ont
TRIMMED_FASTQ=$DATA_DIR/trimmed.fastq.gz
BARCODES=$DATA_DIR/barcodes.txt

python extract_barcodes.py $BARCODES $TRIMMED_FASTQ $DATA_DIR/barcodes.fastq.gz
```

--------

File name: 03_barcode_fastq.sh